In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle
import matplotlib
import ipyparams

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import func_optimize as fo
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
import neurolib.dashboard.layout as layout
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

cmap_ = layout.getcolormap()
darkgrey, midgrey, lightgrey, color_bi_updown, color_LC, color_bi_uposc = layout.getcolors_rgba()
exc_red = cmap_(3)
inh_blue=cmap_(0)
green = cmap_(2)

<IPython.core.display.Javascript object>

not here


In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

def rep_ind(c0, c1):
    abs_diff = np.abs(c0-c1)
    res = 0.
    for n in range(c0.shape[0]):
        for v in range(c0.shape[1]):
            res += sum(abs_diff[n,v,:]) * dt
            
    return res

In [3]:
points = np.array( [ [0.4, 0.322] ])

# 10 points, two costs, 4 methods, N 

In [4]:
def create_dict(task, point, cost_params, init, init_vars, target, sigma_array, N_max_array, it_array):
    return dict( {'task':task, 'point':point, 'cost_params':cost_params, 'init':init, 'init_vars':init_vars,
            'target':target, 'sigma':sigma_array, 'N_array':N_max_array, 'it_array':it_array,
            'cost':np.zeros(( len(sigma_array), 4, 2, len(N_max_array), len(it_array) )),
            'rep_ind':np.zeros(( len(sigma_array), 4, 2, len(N_max_array), len(it_array) ))     } )

params_list = ['task', 'point', 'cost_params', 'init', 'init_vars', 'target', 'sigma', 'N_array',
               'cost', 'rep_ind']

In [5]:
dict_list = []
sigma_array = np.arange(0.01, 0.031, 0.01)
N_array = [2,10,30,50]
it_array = np.arange(0,25,6)

for p in points:
    dict_list.append(create_dict('DU', p, [None]*len(sigma_array), [1], None,
                [None]*len(sigma_array), sigma_array, N_array, it_array) )

In [6]:
for d in dict_list:
    print(d['target'])

[None, None, None]


In [7]:
dict_file = "OU_1I.pickle"

if False:
    file=open(dict_file, "wb")
    pickle.dump(dict_list, file)
    file.close()

if True:
    file = open(dict_file, "rb")
    dict_list = pickle.load(file)
    file.close()

In [8]:
dur_pre = 10
dur_post = 10
dur = 500.
dur_comp = 2000.
dt = aln.params.dt
T = int(1 + dur / dt)

prec_vars = [0,1]
trans_time = 0.8

n_pre = int(np.around(dur_pre/aln.params.dt,1))
n_post = int(np.around(dur_post/aln.params.dt,1))

tol = 1e-16
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

maxC = [5., -5., 0.18, 0.]
n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

noise_realizations_final = 10000
noise_realizations_target_compute = 1000
init_it = 3

In [9]:
# get init vars without noise
aln = ALNModel()
data.set_parameters(aln)
aln.params.duration = dur_comp

# initial without noise
control0 = aln.getZeroControl()

for d in dict_list:
    
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    aln.params.duration = dur_comp
    
    if type(d['init_vars']) == type(None):
    
        control0[0,0,:500] = -5.

        aln.run(control=control0)

        init_state_vars = np.zeros(( len(state_vars) ))
        for j in range(len(state_vars)):
            if aln.state[state_vars[j]].size == 1:
                init_state_vars[j] = aln.state[state_vars[j]][0]
            else:
                init_state_vars[j] = aln.state[state_vars[j]][0,-1]

        initVars = init_state_vars.copy()
        d['init_vars'] = initVars
    
    for ind_s in range(len(d['sigma'])):
        
        if type(d['target'][ind_s]) != type(None):
            continue        
        
        s = d['sigma'][ind_s]
        aln.params.sigma_ou = s
        aln.params.duration = dur_comp
        control0[0,0,:500] = 5.
        
        stateswitchcount = 0
        target_rates_e = []
        target_rates_i = []

        for j in range(noise_realizations_target_compute):
            if j % 100 == 0:
                print(j)
            aln.run(control=control0)
            #plt.plot(aln.t, aln.rates_exc[0,:])
            if np.amin(aln.rates_exc[0,1000:]) < 1.:
                #print("fell in down state, repeat")
                j -= 1
                stateswitchcount += 1
                if stateswitchcount > 0.8 * noise_realizations_target_compute:
                    print("UNSTABLE: point, sigma ", d['point'], s)
                    break
                continue
            target_rates_e.append(aln.rates_exc[0,1000:])
            target_rates_i.append(aln.rates_inh[0,1000:])
        
        #plt.show()            

        aln.params.duration = dur
        target = aln.getZeroTarget()
        target[:,0,:] = np.mean(target_rates_e)
        target[:,1,:] = np.mean(target_rates_i)
        stddev = np.std(target_rates_e) + np.std(target_rates_i)
        
        d['target'][ind_s] = target
        d['cost_params'][ind_s] = [1. / ( stddev * (1. - trans_time) * dur ), 0., 1.]
        
        #print(stddev)
        
    file=open(dict_file, "wb")
    pickle.dump(dict_list, file)
    file.close()

Point =  [0.4   0.322]


In [16]:
for d in dict_list:
     for s_ind in range(len(d['sigma'])):
            for i_it in range(1,len(d['it_array'])):
                for n_ind in range(len(d['N_array'])):
                    if np.isnan(d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it]):
                        print(s_ind, i_it, n_ind)
                        d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = 0.
                    if np.isnan(d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it]):
                        print(s_ind, i_it, n_ind)
                        d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = 0.

0 4 2
1 3 3
2 4 3


In [39]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 1
method_ind = 0
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        aln.params.duration = dur
        zero_init = aln.getZeroControl()
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
        
        if np.amin(d['cost'][s_ind, method_ind, step_method_ind, :, 0]) == 0.:
        
            setinit(d['init_vars'], aln)
            cost_final = 0.

            for n_ in range(noise_realizations_final):
                state_ = fo.updateState(aln, zero_init)
                cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                               state_, target_, list(prec_vars) )

            d['cost'][s_ind, :, :, :, 0] = cost_final/noise_realizations_final
            
        print('cost = ', d['cost'][s_ind, 0, 0, 0])    
        
        for i_it in range(1,len(d['it_array'])):
            
            it = d['it_array'][i_it]
            print("iteration = ", it)
        
            for n_ind in range(len(d['N_array'])):

                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                    
                n_max = d['N_array'][n_ind]
                print("n max = ", n_max)

                aln.params.duration = dur         
                    
                n_ = 0
                control_sum = aln.getZeroControl()
                
                while n_ < n_max:
                
                    print('--------', n_)

                    aln.params.duration = dur
                    control_init = aln.getZeroControl()           
                    setinit(d['init_vars'], aln)
                    
                    c_vars = d['init']

                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_init = bc[:,:,n_pre:-n_post]
                    c_vars = [0,1]
                    aln.params.duration = dur
                    setinit(d['init_vars'], aln)
                    
                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = it-init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_sum += bc[:,:,n_pre:-n_post]
                    n_ += 1

                control_init = control_sum / n_max
        
                aln.params.duration = dur
                setinit(d['init_vars'], aln)

                cost_final = 0.

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )
                    cost_final += cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2],
                                                                   dt, control_init)

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = cost_final/noise_realizations_final
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                ###############################################################
                
                aln.params.duration = dur
                setinit(d['init_vars'], aln)         
                    
                n_ = 0
                control_sum = aln.getZeroControl()
                
                while n_ < n_max:
                
                    print('--------', n_)

                    aln.params.duration = dur
                    control_init1 = aln.getZeroControl()           
                    setinit(d['init_vars'], aln)
                    
                    c_vars = d['init']

                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_init1 = bc[:,:,n_pre:-n_post]
                    c_vars = [0,1]
                    aln.params.duration = dur
                    setinit(d['init_vars'], aln)
                    
                    bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                        control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = it-init_it,
                        tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                        t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                        control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                        noise_real=0, init_params = d['init_vars'])
                    
                    if np.isnan(c_).any():
                        continue
                    
                    control_sum += bc[:,:,n_pre:-n_post]
                    n_ += 1

                control_init1 = control_sum / n_max
                
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                                
    break

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008337123009840281 0.0 1.0
cost =  [1392.03372946  944.34724965  886.47224405  953.59810777  485.9153699 ]
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.02
set cost params:  0.0041585227137248 0.0 1.0
cost =  [643.10760131  44.36820614   8.42323645 466.57144967  52.90775282]
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.0027474834877551967 0.0 1.0
cost =  [281.95557574  18.93011837  91.42430686 214.27640011 137.96201803]
iteration =  6
iteration =  12
iteration =  18
iteration =  24


In [40]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 2
method_ind = 1
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):

        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.:
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M2')

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008337123009840281 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.02
set cost params:  0.0041585227137248 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.0027474834877551967 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration

In [34]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 2
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                                
                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3')

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008337123009840281 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1385.5178082628142
Gradient descend method:  None
RUN  1 , total integrated cost =  112.42654002859193
step size too small or too large
RUN  2 , total integrated cost =  16.24473373178873
RUN  3 , total integrated cost =  1.8003876150444762
RUN  3 , total integrated cost mean =  1.8003876150444762
Improved over  3  iterations in  15.990485548973083  seconds by  99.87005669618192  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  1.8313669225122458
Gradient descend method:  None
RUN  1 , total integrated cost =  1.5643222126044773
step size too small or too large
RUN  2 , total integrated cost =

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size 

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size 

RUN  1 , total integrated cost =  1.4803562587217243
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  2 , total integrated cost =  1.4751276325676388
step size too small or too large
step size too small or too large
step size too small or too large
RUN  3 , total integrated cost =  1.4764772833594781
RUN  3 , total i

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size 

...

step size too small or too large
step size too small or too large
RUN  13 , total integrated cost =  5.230437876358498
RUN  13 , total integrated cost mean =  5.230437876358498
Improved over  13  iterations in  161.63200214400422  seconds by  91.59660404759343  percent.
rep_ind =  52.10061882949468
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  295.5208522956492
Gradient descend method:  None
step size too small or too large
RUN  1 , total integrated cost =  108.08112120949743
RUN  2 , total integrated cost =  30.442692182583098
step size too small or too large
step size too small or too large
RUN  3 , total integrated cost =  149.31428737732347
RUN  3 , total integrated cost mean =  149.31428737732347
Improved over  3  iterations in  12.38537167198956  seconds by  49.474195740358674  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  132.24095024489137
Gradient descend method:  None
step size too small or too large

step size too small or too large
RUN  1 , total integrated cost =  2.59504125344498
step size too small or too large
RUN  2 , total integrated cost =  2.1498007845725953
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step 

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  4 , total integrated cost =  41.120908436217775
step size too small or too large
step size too small or too large
RUN  5 , total integrated cost =  29.64068348715062
no cost improvement
RUN  6 , total integrated cost =  4.815909702150269
step size too small or too large
step size too small or too larg

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size 

step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
step size too small or too large
RUN  10 , total integrated cost =  2.3198665658617212
step size too small or too large
step size too small or too large
RUN  11 , total integrated cost =  5.5240306788055555
no cost improvement
step size too small or too large
step size too small or too large
step size too 

In [41]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 4
method_ind = 3
step_method_ind = 0

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4')
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4')

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4')
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4')

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008337123009840281 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.02
set cost params:  0.0041585227137248 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.0027474834877551967 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration

In [23]:
for s_ind in range(len(d['sigma'])):
    for n_ind in range(len(d['N_array'])):
        for i_it in range(1,len(d['it_array'])):
            #d['cost'][2, 2, 0, n_ind, i_it] = 0.

In [42]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 2
step_method_ind = 1
step_m = 'S1'
if step_method_ind == 1:
    step_m = 'S2'

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M3', method_step=step_m)

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()
                
    break

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008337123009840281 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.02
set cost params:  0.0041585227137248 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.0027474834877551967 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration

In [ ]:
#len(sigma_array), 4, N_max, max_it

##### METHOD 3
method_ind = 3
step_method_ind = 1
step_m = 'S1'
if step_method_ind == 1:
    step_m = 'S2'

for d in dict_list:
    print("Point = ", d['point'])
    aln.params.ext_exc_current = d['point'][0] * 5.
    aln.params.ext_inh_current = d['point'][1] * 5.
    
    max_it = d['cost'].shape[3]
    
    for s_ind in range(len(d['sigma'])):
        
        print("sigma = ", d['sigma'][s_ind])
        
        s = d['sigma'][s_ind]
        aln.params.sigma_ou = s
        c_p = d['cost_params'][s_ind]
        cost.setParams(c_p[0], c_p[1], c_p[2])
        
        target_ = d['target'][s_ind].copy()
        target_[0,0,:int(trans_time * dur / aln.params.dt)] = -1000
        target_[0,1,:int(trans_time * dur / aln.params.dt)] = -1000
                
        for n_ind in range(len(d['N_array'])):
            
            n_ = d['N_array'][n_ind]
            
            print("n = ", n_)
            aln.params.duration = dur
        
            for i_it in range(1,len(d['it_array'])):
            
                it = d['it_array'][i_it]
                print("iteration = ", it)
                
                if ( d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0.
                    and d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] > 0. ) :
                    continue
                
                c_vars = d['init']
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                control_init = aln.getZeroControl()
                    
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
            
                control_init = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)

                control_init = bc[:,:,100:-100]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                cost_final = 0.
                cs = cost.numba_cost_sparsity_int(1, 6, T, d['cost_params'][s_ind][2], dt, control_init)

                for j in range(noise_realizations_final):
                    state_ = fo.updateState(aln, control_init)
                    cost_final += cost.numba_cost_precision_int(1, T, dt, d['cost_params'][s_ind][0],
                                                                   state_, target_, list(prec_vars) )

                d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it] = (
                                cost_final / noise_realizations_final + cs )
                print('cost = ', d['cost'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                c_vars = d['init']                    
                aln.params.duration = dur
                control_init1 = aln.getZeroControl()
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_ = init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)
            
                control_init1 = bc[:,:,100:-100]
                c_vars = [0,1]
                aln.params.duration = dur
                setinit(d['init_vars'], aln)
                
                bc, bs, c_, runtime, grad, phi, costnode, total_cost_mean_std = aln.A1(
                    control_init1, d['target'][s_ind], c_scheme, u_mat, u_scheme, max_iteration_=it-init_it,
                    tolerance_ = tol, startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl,
                    t_sim_ = dur, t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = None,
                    control_variables_ = c_vars, prec_variables_ = prec_vars, transition_time_ = trans_time,
                    noise_real=n_, init_params = d['init_vars'], method='M4', method_step=step_m)

                control_init1 = bc[:,:,100:-100]
                d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it] = rep_ind(control_init, control_init1)
                print('rep_ind = ', d['rep_ind'][s_ind, method_ind, step_method_ind, n_ind, i_it])
                
                file=open(dict_file, "wb")
                pickle.dump(dict_list, file)
                file.close()

Point =  [0.4   0.322]
sigma =  0.01
set cost params:  0.008337123009840281 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.02
set cost params:  0.0041585227137248 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  10
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  30
iteration =  6
iteration =  12
iteration =  18
iteration =  24
n =  50
iteration =  6
iteration =  12
iteration =  18
iteration =  24
sigma =  0.03
set cost params:  0.0027474834877551967 0.0 1.0
n =  2
iteration =  6
iteration =  12
iteration =  18
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  223.77757574870992
Gradient descend method:  None
RUN  1 , total integrated cost =  2.2635179448802036


RUN  1 , total integrated cost =  4.739250848312423
RUN  2 , total integrated cost =  101.70842384379691
RUN  3 , total integrated cost =  103.02088406438133
RUN  3 , total integrated cost mean =  103.02088406438133
Improved over  3  iterations in  1.9673636879306287  seconds by  48.028045878047166  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  18.38645331597064
Gradient descend method:  None
RUN  1 , total integrated cost =  101.265914739252
RUN  2 , total integrated cost =  100.44823472229935
RUN  3 , total integrated cost =  100.00172927831818
RUN  4 , total integrated cost =  101.15655803787078
RUN  5 , total integrated cost =  117.4799866891703
no cost improvement
RUN  6 , total integrated cost =  100.36315017465584
RUN  7 , total integrated cost =  99.77621340032998
RUN  7 , total integrated cost mean =  99.77621340032998
Improved over  7  iterations in  9.49492776102852  seconds by  -442.6615546004375  percent.
cost =  18.182372899596924
interp

RUN  1 , total integrated cost =  46.80001228693023
RUN  2 , total integrated cost =  45.94576099055078
RUN  3 , total integrated cost =  45.21386788711736
RUN  3 , total integrated cost mean =  45.21386788711736
Improved over  3  iterations in  6.98079346108716  seconds by  -1405.8978151024037  percent.
cost =  3.0804067489937856
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  306.1937569416225
Gradient descend method:  None
RUN  1 , total integrated cost =  482.4129456993922
RUN  2 , total integrated cost =  52.82050290682642
RUN  3 , total integrated cost =  48.37140063027585
RUN  3 , total integrated cost mean =  48.37140063027585
Improved over  3  iterations in  2.4181477279635146  seconds by  84.20235568699133  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  14.496441363976606
Gradient descend method:  None
RUN  1 , total integrated cost =  47.72891401216291
RUN  2 , total integrated cost =  41.93009254261809
RUN  3 , total

RUN  1 , total integrated cost =  14.325759991870301
RUN  2 , total integrated cost =  834.0733150245868
RUN  3 , total integrated cost =  1252.6421986089451
RUN  3 , total integrated cost mean =  1252.6421986089451
Improved over  3  iterations in  2.416559719014913  seconds by  -476.58553385119524  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  75.88225778049522
Gradient descend method:  None
RUN  1 , total integrated cost =  1174.6950739884157
RUN  2 , total integrated cost =  1380.4566368720918
RUN  3 , total integrated cost =  1298.150519383551
RUN  4 , total integrated cost =  1639.772332497664
RUN  5 , total integrated cost =  1639.742982301004
no cost improvement
RUN  6 , total integrated cost =  1639.8127728212125
step size too small or too large
step size too small or too large
RUN  7 , total integrated cost =  105.65918605699625
Control only changes marginally.
RUN  7 , total integrated cost mean =  105.65918605699625
Improved over  7  iterat

rep_ind =  61.371428121919344
iteration =  24
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  257.24008724376523
Gradient descend method:  None
RUN  1 , total integrated cost =  280.8740006413893
RUN  2 , total integrated cost =  185.2115721068165
RUN  3 , total integrated cost =  288.4214621896736
RUN  3 , total integrated cost mean =  288.4214621896736
Improved over  3  iterations in  7.603940363973379  seconds by  -12.121506908198313  percent.
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  5.571003898616905
Gradient descend method:  None
RUN  1 , total integrated cost =  83.14484233932598
RUN  2 , total integrated cost =  90.77439144857627
RUN  3 , total integrated cost =  87.74651550478022
RUN  4 , total integrated cost =  83.86439018955663
RUN  5 , total integrated cost =  83.64640708520268
RUN  6 , total integrated cost =  81.85947293514188
RUN  7 , total integrated cost =  83.28296511771423
RUN  8 , total integrated cost =  82.289